In [8]:
import sys
assert sys.version_info >= (3, 8)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.

In [10]:
import imutils
from utils import display_image

# Load the image
image = cv.imread('images/lena.jfif')

# Strategy 1: Using imutils
rotated_imutils = imutils.rotate_bound(image, 45)
display_image("Rotated Image using imutils", rotated_imutils, adjust=True)

# Strategy 2: Modifying the transformation matrix
(h, w) = image.shape[:2]
(cX, cY) = (w // 2, h // 2)
M = cv.getRotationMatrix2D((cX, cY), 45, 1.0)
cos = np.abs(M[0, 0])
sin = np.abs(M[0, 1])
new_w = int((h * sin) + (w * cos))
new_h = int((h * cos) + (w * sin))
M[0, 2] += (new_w / 2) - cX
M[1, 2] += (new_h / 2) - cY
rotated_matrix = cv.warpAffine(image, M, (new_w, new_h))
display_image("Rotated Image using transformation matrix", rotated_matrix, adjust=True)


2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")

In [12]:
# 2
flower_img = cv.imread("images/flower.jfif")
bee_img = cv.imread("images/native-bee.png")

# location of flower on top of bee's head
rows, cols, channels = flower_img.shape
roi = bee_img[0:rows, 0:cols]

# mask of flower and create inverse mask of flower
flower_gray = cv.cvtColor(flower_img, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(flower_gray, 74, 255, cv.THRESH_BINARY)
inv_mask = cv.bitwise_not(mask)

# black-out the area of flower in ROI
bee_bg = cv.bitwise_and(roi, roi, mask = inv_mask)

# extract only region of flower from the flower_img
flower_fg = cv.bitwise_and(flower_img, flower_img, mask = mask)

# put flower in ROI and modify the main image
dst = cv.add(bee_bg, flower_fg)
bee_img[0:rows, 0:cols] = dst

# Display result
cv.imshow('Result', bee_img)
cv.waitKey(0)
cv.destroyAllWindows()

3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

In [16]:
import random
from utils import display_image

def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    h, w = image.shape[:2]
    assert 0 < min_crop_ratio <= max_crop_ratio < 1, "Crop ratios should be between 0 and 1"
    crop_ratio = random.uniform(min_crop_ratio, max_crop_ratio)
    crop_h = int(h * crop_ratio)
    crop_w = int(w * crop_ratio)
    start_y = (h - crop_h) // 2
    start_x = (w - crop_w) // 2
    cropped_image = image[start_y:start_y+crop_h, start_x:start_x+crop_w]
    return cropped_image

# Example usage
image = cv.imread('images/lena.jfif')
cropped_image = random_center_crop(image, 0.5, 0.8)
display_image("Cropped Image", cropped_image, adjust=True)


4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [20]:
from utils import display_image

def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    noisy_image = np.copy(image)
    num_salt = np.ceil(salt_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    noisy_image[coords[0], coords[1], :] = 1
    num_pepper = np.ceil(pepper_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    noisy_image[coords[0], coords[1], :] = 0
    return noisy_image

# Example usage
image = cv.imread('images/lena.jfif')
noisy_image = add_salt_and_pepper_noise(image, 0.02, 0.02)
display_image("Noisy Image", noisy_image, adjust=True)
